# Example 5: Extract Class

[Extract class in the refactoring catalog.](http://refactoring.com/catalog/extractClass.html)

- Do this when a class is responsible for too many things (> 5 methods, > 5 properties)
- Define a new class with all the fields you need to do the work.
- Implement `@property` and `@property.setter` to point at the new helper class so old usage continues to work.
- Move the methods that interact with these properties onto the interior class, instead of on the exterior class. Use `@property` to point existing usage at the new helper class.
- Use warnings to track down old usage. Move that usage to directly interact with the interior class. Remove all the `@property`s you used for indirection.